# BentoML Demo: Iris Classifier with custom web UI

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. 

This notebook demonstrates how to use BentoML to __serve a Iris Classification model containing a REST API server with Custom Web UI__.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=h2o&ea=h2o-loan-default-prediction&dt=h2o-loan-default-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install -q bentoml
!pip install -q sklearn

## Create BentoService for model serving

In [2]:
%%writefile iris_classifier.py
from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.artifact import SklearnModelArtifact

@env(auto_pip_dependencies=True)
@artifacts([SklearnModelArtifact('model')])
class IrisClassifier(BentoService):

    @api(input=DataframeInput())
    def predict(self, df):
        # Optional pre-processing, post-processing code goes here
        return self.artifacts.model.predict(df)

Overwriting iris_classifier.py


In [3]:
%%writefile main.py
from sklearn import svm
from sklearn import datasets

from iris_classifier import IrisClassifier

if __name__ == "__main__":
    # Load training data
    iris = datasets.load_iris()
    X, y = iris.data, iris.target

    # Model Training
    clf = svm.SVC(gamma='scale')
    clf.fit(X, y)

    # Create a iris classifier service instance
    iris_classifier_service = IrisClassifier()

    # Pack the newly trained model artifact
    iris_classifier_service.pack('model', clf)

    # Save the prediction service to disk for model serving
    saved_path = iris_classifier_service.save()

Overwriting main.py


In [4]:
!python main.py

[2020-07-30 09:05:23,280] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-07-30 09:05:34,048] INFO - Detect BentoML installed in development model, copying local BentoML module file to target saved bundle path
running sdist
running egg_info
writing BentoML.egg-info/PKG-INFO
writing dependency_links to BentoML.egg-info/dependency_links.txt
writing entry points to BentoML.egg-info/entry_points.txt
writing requirements to BentoML.egg-info/requires.txt
writing top-level names to BentoML.egg-info/top_level.txt
reading manifest file 'BentoML.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'
writing manifest file 

## Model Serving via REST API

Start local API model server with `bentoml serve` command:

In [6]:
!bentoml serve IrisClassifier:latest

[2020-07-28 16:38:55,465] INFO - Getting latest version IrisClassifier:20200728163753_DE85E9
[2020-07-28 16:38:56,409] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-07-28 16:38:56,422] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1, but loading from BentoML version 0.8.1+27.g8e155f5.dirty
 * Serving Flask app "IrisClassifier" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
^C


At this point you can test out the Rest API server either by opening http://localhost:5000 in a new tab which will serve the swagger docs:

![alt text](https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/swagger.png)


or by making a curl request in a another terminal window:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[[1,2,1,2]]' \
localhost:5000/predict
```

## Adding Custom Web Static Content


In [7]:
!curl https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/static.tar.xz -o static.tar.xz
!tar --xz -xf static.tar.xz
!rm static.tar.xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  197k  100  197k    0     0   200k      0 --:--:-- --:--:-- --:--:--  200k


Here we have a very simple web ui as our static content that bento will serve.
Now we will edit our bento service to point to this static directory.

Add `@web_static_content('./static')` to `iris_classifier.py`

**Note**: The path can be both relative or absolute. 

In [9]:
%%writefile iris_classifier.py
from bentoml import env, artifacts, api, BentoService, web_static_content
from bentoml.adapters import DataframeInput
from bentoml.artifact import SklearnModelArtifact

@env(auto_pip_dependencies=True)
@artifacts([SklearnModelArtifact('model')])
@web_static_content('./static')
class IrisClassifier(BentoService):

    @api(input=DataframeInput())
    def test(self, df):
        # Optional pre-processing, post-processing code goes here
        return self.artifacts.model.predict(df)

Overwriting iris_classifier.py


In [10]:
!python main.py

[2020-07-30 09:09:28,868] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-07-30 09:09:39,344] INFO - Detect BentoML installed in development model, copying local BentoML module file to target saved bundle path
running sdist
running egg_info
writing BentoML.egg-info/PKG-INFO
writing dependency_links to BentoML.egg-info/dependency_links.txt
writing entry points to BentoML.egg-info/entry_points.txt
writing requirements to BentoML.egg-info/requires.txt
writing top-level names to BentoML.egg-info/top_level.txt
reading manifest file 'BentoML.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'
writing manifest file 

In [ ]:
!bentoml serve IrisClassifier:latest

[2020-07-30 09:09:43,808] INFO - Getting latest version IrisClassifier:20200730090929_C02FB7
[2020-07-30 09:09:43,808] INFO - Starting BentoML API server in development mode..
[2020-07-30 09:09:44,916] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-07-30 09:09:44,949] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.3, but loading from BentoML version 0.8.3+42.gb8d36b6
 * Serving Flask app "IrisClassifier" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jul/2020 09:09:52] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2020 09:09:55] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2020 09:09:57] "PO

Now if you visit http://localhost:5000/, you should be served with a beautiful UI:

![Custom UI](https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/webui.png)

It's still possible to access the swagger docs at `/docs`